In [32]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings('ignore')

In [24]:
pip install pandas==2.1.1 numpy==2.0.0

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     -------------------------- ------------- 41.0/60.9 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 60.9/60.9 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.4/10.6 MB 13.2 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/10.6 MB 12.2 MB/s eta 0:00:01
   ---- ----------------------------------- 1.3/10.6 MB 10.5 MB/s eta 0:00:01
   ------ --------------------------------- 1.6/10.6 MB 10.4 MB/s eta 0:00:01
   ------ --------------------------------- 1.6/10.6 MB 10.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.2/10.6 MB 8.8 MB/s eta 0:00:01
   ---------- ----------------------------- 2.7/10.6 MB 8.5 MB/s eta 0:00:01
   ------------ --------------------------- 3.3/10.6 MB 9.2 MB/s eta 0:00:01
   ------------- -------------------------- 3.6/10.6 MB 8.9 MB/s eta 0:00:01


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
pip install numpy==1.15.1

     ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.5 MB 660.6 kB/s eta 0:00:07
     - -------------------------------------- 0.2/4.5 MB 3.1 MB/s eta 0:00:02
     ----- ---------------------------------- 0.6/4.5 MB 5.5 MB/s eta 0:00:01
     --------- ------------------------------ 1.1/4.5 MB 6.2 MB/s eta 0:00:01
     ------------ --------------------------- 1.4/4.5 MB 6.2 MB/s eta 0:00:01
     --------------- ------------------------ 1.7/4.5 MB 6.4 MB/s eta 0:00:01
     ---------------- ----------------------- 1.9/4.5 MB 6.6 MB/s eta 0:00:01
     ------------------ --------------------- 2.1/4.5 MB 5.8 MB/s eta 0:00:01
     ------------------ --------------------- 2.1/4.5 MB 5.8 MB/s eta 0:00:01
     --------------------- ------------------ 2.4/4.5 MB 5.6 MB/s eta 0:00:01
     ------------------------ --------------- 2.7/4.5 MB 5.6 MB/s eta 0:00:01
     ---------------------------- ----------- 3.2/4.5 MB 6.0 MB/s eta

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [267 lines of output]
      Running from numpy source directory.
      <string>:378: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
      C:\Users\henri\AppData\Local\Temp\pip-install-gxf0yzo3\numpy_f13916cba7134a3b9a090d3b253e0328\numpy\distutils\misc_util.py:474: SyntaxWarning: "is" with a literal. Did you mean "=="?
        return is_string(s) and ('*' in s or '?' is s)
      C:\Users\henri\AppData\Local\Temp\pip-build-env-1b7r_83m\overlay\Lib\site-packages\setuptools\_distutils\dist.py:268: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      blas_opt_info:
      blas_mkl_info:
      No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
      customize MSVCCompiler
        libraries mkl_rt not found in ['c:\\Users\\henri

In [37]:
import pandas as pd
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [4]:
df = pd.read_parquet('./features.parquet')

In [5]:
X = df.iloc[:,1:]

y = df['Churn']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Logistic Regression

In [9]:
encoder = ce.BinaryEncoder(cols=X_train.columns.tolist())

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

In [10]:
model = LogisticRegression()

model.fit(X_train, y_train)

LogisticRegression()

In [11]:
y_pred_test = model.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

Model accuracy score: 0.7722


In [13]:
print('Training set score: {:.4f}'.format(model.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

Training set score: 0.7934
Test set score: 0.7722


# Catboost

clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    #loss_function='CrossEntropy'
)


clf.fit(X_train, y_train, 
        cat_features=X.columns.tolist(), 
        eval_set=(X_train, y_train), 
        verbose=False
)